# Notebook for fine mapping based on the PD meta5v2 summary results

## Jan 27, 2020
## **Author** - Raph Gibbs

#### set global variables and import libraries

In [ ]:
#set up notebook global variables
WRKDIR = '$PATH/spd/finemap/meta5v2'
AUTOSOMES = [str(x) for x in list(range(1,23))]
SEXOMES = ['X','Y']
CHROMOSOMES = AUTOSOMES + SEXOMES

INDEX_FLANK_SIZE = 1000000
# INDEX_FLANK_SIZE = 500000
MYUSER = ''
GWAS_SIG_PVALUE_THRESHOLD = 5e-08

In [ ]:
#import libraries
import threading
import modin.pandas as pd
import dask.dataframe as dd
import pandas
import seaborn as sns
import numpy as np
import os.path
import matplotlib.pyplot as plt

# import pandas as pd
# from tqdm import tqdm
# # import math
# # import numba
# # from umap import UMAP
# # import scipy.stats as stats
# # import statsmodels.stats.multitest as smm
# # import time
# from numba import jit

# import warnings
# warnings.filterwarnings("ignore")

#### do the simple thing first, ie using the IPDGC PD meta5v2 summary stats, use more heavily filtered summary stats from CARD (Cornelis pointed me to) path which only contained rsIDs and since had to annotate positions went ahead and used hg38 which makes using TOPMed reference panel since that is in hg38 with postion style variant names

In [ ]:
#pull the files local for easier manipulation

#path to hold IPDGC meta5v2 original files
!mkdir -p {WRKDIR}

print('\n#run these commands at terminal:\n')

summary_results_for_smr = '$PATH/PD/summary_stats/resultsForSmr_filtered.tab.gz'
print(f'scp {MYUSER}@helix.nih.gov:{summary_results_for_smr} {WRKDIR}/')

#### load the summary stats

In [ ]:
# load the summary stats
summary_stats_df = pd.read_csv(f'{WRKDIR}/resultsForSmr_filtered.tab.gz', sep='\t')
print(summary_stats_df.shape)
summary_stats_df.head()

In [ ]:
#for some reason this files has duplicates, drop them
summary_stats_df = summary_stats_df.drop_duplicates(subset=['SNP'])

print(summary_stats_df.shape)
summary_stats_df.head()

#### need to map dbSNP rsIDs to reference build positions

In [ ]:
#create limited target function for threading, just run the bash majic
def run_cmd_by_majic(the_cmd):
    !{the_cmd}

#funciton to pull down the UCSC dbSNP files
def pull_ucsc_dbsnp_files(version_build_name, chrs_list, out_dir):
    snp_bed_url = f'ftp://ftp.ncbi.nih.gov/snp/organisms/{version_build_name}/BED/'
    
    job_threads = []
    for chrom in chrs_list:
        this_chr_snp_bed = f'bed_chr_{chrom}.bed.gz'
        snp_bed_file = snp_bed_url + this_chr_snp_bed
        this_cmd = f'curl --silent -L {snp_bed_file} --output {out_dir}/{this_chr_snp_bed}'
#         print(this_cmd)
#         !{this_cmd}
        this_thread = threading.Thread(target=run_cmd_by_majic, args=(this_cmd,))
        job_threads.append(this_thread)
        this_thread.start()
    
    for job_thread in job_threads:
        job_thread.join()

In [ ]:
# pull down hg38 dbSNP b151
this_out_dir = f'{WRKDIR}/tools/hg38'
!mkdir -p {this_out_dir}

# pull_ucsc_dbsnp_files('human_9606_b151_GRCh38p7', AUTOSOMES, this_out_dir)
pull_ucsc_dbsnp_files('human_9606_b151_GRCh38p7', CHROMOSOMES, this_out_dir)

In [ ]:
# pull down GRCh37 dbSNP b151
this_out_dir = f'{WRKDIR}/tools/hg19'
!mkdir -p {this_out_dir}

# pull_ucsc_dbsnp_files('human_9606_b151_GRCh37p13', AUTOSOMES, this_out_dir)
pull_ucsc_dbsnp_files('human_9606_b151_GRCh37p13', CHROMOSOMES, this_out_dir)

In [ ]:
#function to load each chrom's known variants, subset by variants of interest and append
#the skiprows argument is has a different effect between modin, dask.dataframe and pandas, \
#have to use default pandas here instead of modin.pandas

def find_dbsnp_positions(var_list, chrs_list, build_path):
    snp_bed_df = None
    for chrom in chrs_list:
        print(chrom, end='.')
        snp_chr_bed = pandas.read_csv(f'{build_path}/bed_chr_{chrom}.bed.gz', \
                                  header=None, skiprows=1, sep='\s+')

        this_chr = snp_chr_bed.loc[snp_chr_bed[3].isin(var_list),[0, 2, 3]]

        snp_bed_df = pandas.concat([snp_bed_df, this_chr])

    snp_bed_df.columns = ['chr', 'position', 'id']    
    
    return snp_bed_df

In [ ]:
%%time
#now pull and map ids for index variants
build = 'hg38'
build_path = f'{WRKDIR}/tools/{build}'

#all the variants of interest are proximal to LRRK2 on chr12
var_positions_df = find_dbsnp_positions(list(summary_stats_df['SNP']), \
                                             AUTOSOMES, build_path)
print(var_positions_df.shape)
var_positions_df.head()

In [ ]:
#so looks like some number of summary stats rsIDs are not present in dbSNP151
missing_rsIDs = set(summary_stats_df['SNP']) - set(var_positions_df['id'])
print(len(missing_rsIDs))
#33K different in size but only 13K missing lots of dupes?
sum_stats_missing_pos_df = summary_stats_df.loc[summary_stats_df['SNP'].\
                                                isin(list(missing_rsIDs))]

print(sum_stats_missing_pos_df.shape)
print(sum_stats_missing_pos_df['p'].describe())
print(sum_stats_missing_pos_df.loc[sum_stats_missing_pos_df['p'] < 1e-08].shape)

In [ ]:
# the dbSNP IDs are no longer valid but had significant results
sum_stats_missing_pos_df.loc[sum_stats_missing_pos_df['p'] < 1e-08]

In [ ]:
# merge summary stats with variants physical position
summary_stats_df = summary_stats_df.merge(pd.DataFrame(var_positions_df), \
                                          how='inner', left_on='SNP', right_on='id')

summary_stats_df.head()

#### interim save point

In [ ]:
%%time
# go head and do an interim save, for ease of reloading
this_h5_file = f'{WRKDIR}/pdmeta_sumstats_hg38.h5'

# reload_df = False
reload_df = True

if reload_df:
    print(f'loading {this_h5_file}')
    summary_stats_df = pd.read_hdf(this_h5_file)
else:
    print(f'saving {this_h5_file}')
    summary_stats_df.to_hdf(this_h5_file, key='summarystats', mode='w')

#### split results into regions

In [ ]:
#small function to extract (subset and remove) specified regions summary stats
def extract_stats_by_region(source_df, start_bp, stop_bp):
    # subset the regions results
    result_df = source_df.loc[(source_df['position'] >= start_bp) & \
                              (source_df['position'] <= stop_bp)]
    # now remove remove the region from the source 
    #for some reason, guess referencing, this doesn't work from this function
    source_df.drop(result_df.index, inplace=True)
    return result_df


#function to pull, check, and merge regions when needed
def find_regions_for_chrom(stats_df, chrom_str):
    #buffer to add of offset and avoid any silly small gaps that might show up
    #instead manually setting this buffer make 10% of flank size constant
    buffer_size = int(INDEX_FLANK_SIZE * 0.1)

    # split results by physical distance, using 1Mb as it what finemap uses in their paper
    stats_to_parse = stats_df.loc[stats_df['chr'] == chrom_str].copy()

    chroms_regions = []
    #get the first row, ie smallest p-value and pull all proximal
    while True:
        stats_to_parse.sort_values(by='p', inplace=True)
        top_row = stats_to_parse.iloc[0]

        if top_row['p'] > GWAS_SIG_PVALUE_THRESHOLD:
            break

        # extract (with deletion of rows) index and flanking variants
        this_bp_min = top_row['position'] - INDEX_FLANK_SIZE
        this_bp_max = top_row['position'] + INDEX_FLANK_SIZE
        this_region = extract_stats_by_region(stats_to_parse, this_bp_min, this_bp_max)

        #if no regions for this chrom add, otherwise, see if part of existing region
        if not chroms_regions:
            chroms_regions.append(this_region)
        else:
            #check that this next top variant isn't just part of an exsiting region
            for list_index, a_region in enumerate(chroms_regions):
                a_min_bp = a_region['position'].min()
                a_max_bp = a_region['position'].max()

                #see if new region overlaps with this enumerated regions
                #ie if max end of region greater that min end or region or vice versa
                if (this_bp_min > (a_max_bp + buffer_size)) | (this_bp_max < (a_min_bp - buffer_size)):
                    #does overlap region at all
                    #if this was last regions to check add as new region
                    if(list_index == len(chroms_regions)-1):
                        chroms_regions.append(this_region)
                        break
                    else:
                        continue
                elif (this_bp_min < (a_max_bp + buffer_size)):
                    #its a downstream adjacent region
                    a_region = pd.concat([a_region, this_region])                
                    chroms_regions[list_index] = a_region
                    break
                elif (this_bp_max > (a_min_bp - buffer_size)):
                    #its an upstream adjacent region
                    a_region = pd.concat([this_region, a_region])
                    chroms_regions[list_index] = a_region    
                    break

    return chroms_regions


In [ ]:
%%time

# retrieve results by chromosomes
#### find chromosomes present in results
chroms_arr = summary_stats_df['chr'].unique()

job_threads = []
regions_by_chrom = {}
for chr_str in chroms_arr:
    print(chr_str, end='.')
    regions_by_chrom[chr_str] = find_regions_for_chrom(summary_stats_df, chr_str)        

# need to sort out parallelizing this bit
#     this_thread = threading.Thread(target=find_regions_for_chrom, \
#                                    args=(summary_stats_df, chr_str))
#     job_threads.append(this_thread)
#     this_thread.start()

# for job_thread in job_threads:
#     job_thread.join()    
    

#### write locus regions to file

In [ ]:
# write locus regions to file
locus_regions_file = f'{WRKDIR}/loci.txt'

with open(locus_regions_file,'w') as regions_file:    
    #write header
    regions_file.write('chrom num_variants index_variant index_p-value start_bp end_bp size_bp\n')  
    for chr_str in chroms_arr:
        for this_df in regions_by_chrom[chr_str]:
            min_bp = this_df['position'].min()
            max_bp = this_df['position'].max()
            top_hits = this_df.loc[this_df['p'] == this_df['p'].min()].iloc[0]
            top_snp = top_hits['SNP']
            smallest_p = top_hits['p']
            regions_str = f'{chr_str} {this_df.shape[0]} {top_snp} {smallest_p:.4E} \
{min_bp} {max_bp} {max_bp-min_bp}'

            regions_file.write(f'{regions_str}\n')        

In [ ]:
# lazy but read that file back in as dataframe
regions_df = pd.read_csv(f'{WRKDIR}/loci.txt', sep='\s+')
print(regions_df.shape)
regions_df.head()

In [ ]:
# for simplicity latter, name the loci/region names
regions_df['name'] = regions_df['chrom'] + '-' + regions_df['start_bp'].astype('str') + \
'-' + regions_df['end_bp'].astype('str')

regions_df.head()

#### extract regoins from reference panel(s) 

In [ ]:
# save a regions to extract file usable by bcftools
regions_file = f'{WRKDIR}/loci_regions.txt'
regions_df[['chrom', 'start_bp', 'end_bp']].to_csv(regions_file, header=False, \
                                                   index=False, sep='\t')

#### QC check for single ancestry imputation reference panel

In [ ]:
#sex check mismatches
def check_sexcheck(path_prefix):
    sexcheck_file = f'{path_prefix}.gender.sexcheck'
    sexcheck_df = pd.read_csv(sexcheck_file, sep='\s+')
    #here failed sexcheck does not include unreported sex
    sexcheck_problems_df = sexcheck_df.loc[(sexcheck_df['STATUS'] == 'PROBLEM') & \
                                           (sexcheck_df['PEDSEX'] != 0)]
    print(f'sex mismatches {sexcheck_problems_df.shape}')
    return sexcheck_problems_df

#check sample missingness 
def check_missingness(path_prefix, max_miss=0.05):
    smiss_file = f'{path_prefix}.missing.smiss'
    #might be plink1 file extenstion
    if not os.path.exists(smiss_file):
        smiss_file = f'{path_prefix}.missing.imiss'
    smiss_df = pd.read_csv(smiss_file, sep='\s+')
    smiss_problems_df = smiss_df.loc[smiss_df['F_MISS'] > max_miss]
    print(f'sample missingness problems {smiss_problems_df.shape}')
    return smiss_problems_df

#het rate problems
def check_hetrate(path_prefix, abs_max_rate=0.15):
    hetcheck_file = f'{path_prefix}.het.het'
    hetcheck_df = pd.read_csv(hetcheck_file, sep='\s+')
    hetcheck_problems_df = hetcheck_df.loc[(hetcheck_df['F'] > abs_max_rate) | \
                                           (hetcheck_df['F'] < -abs_max_rate)]
    print(f'sample het rate problems {hetcheck_problems_df.shape}')
    return hetcheck_problems_df

#sample contamination problems
def check_contamination(path_prefix):
    contam_file = f'{path_prefix}.contaminated.samples.txt'
    #might not exist if didn't process raw data
    if os.path.exists(contam_file):
        contam_df = pd.read_csv(contam_file, sep='\s+')
        print(f'sample contamination problems {contam_df.shape}')
    else:
        print('no comtamination file present')
        contam_df = pd.DataFrame(data=None, columns=['id'])
    return contam_df        

#check ancestry outliers
def check_ancestry(path_prefix):
    pop_outliers_file = f'{path_prefix}.pop.outliers.txt'
    pop_outliers = pd.read_csv(pop_outliers_file, sep='\s+', header=None)
    print(f'sample ancestry outliers {pop_outliers.shape}')
    return pop_outliers

#check sample duplicates (assume no twins)
def check_duplicates(path_prefix):
    relateds_file = f'{path_prefix}.king.related.kin0'
    relateds_df = pd.read_csv(relateds_file, sep='\s+')
    duplicates_df = relateds_df.loc[relateds_df['InfType'] == 'DUP/MZ']
    print(f'sample duplicate pairs {duplicates_df.shape}')
    return duplicates_df


def find_sample_excludes(path_prefix):
    sexcheck_problems_df = check_sexcheck(path_prefix)
    smiss_problems_df = check_missingness(path_prefix)
    hetcheck_problems_df = check_hetrate(path_prefix)
    contam_df = check_contamination(path_prefix)
    pop_outliers = check_ancestry(path_prefix)
    duplicates_df = check_duplicates(path_prefix)
    
    ref_exclude_set = set(sexcheck_problems_df['IID']) | set(smiss_problems_df['IID']) | \
    set(hetcheck_problems_df['IID']) | set(contam_df['id']) | set(pop_outliers[0])

    #note to split duplicate pairs, first break any that currently have QC problems
    #then just grab first column
    duplicates_df = duplicates_df.loc[~duplicates_df['ID1'].isin(list(ref_exclude_set)) & \
                                     ~duplicates_df['ID1'].isin(list(ref_exclude_set))]
    print(f'refreshed duplicate pairs {duplicates_df.shape}')
    
    ref_exclude_set = ref_exclude_set | set(duplicates_df['ID1'])

    print(f'number of samples to exclude from ref panel {len(ref_exclude_set)}')
    return ref_exclude_set

In [ ]:
#generate an excludes list for the TOPMed set
#this of course assumes the files being check in the exclude function exist at \
#this QC path
topmed_qc_prefix = '$PATH/datasets/wgs_ref_haplos/qc/topmed.freeze5b'
topmed_excludes = find_sample_excludes(topmed_qc_prefix)

pd.DataFrame(data=topmed_excludes).to_csv(f'{WRKDIR}/topmed.freeze5b.excludes.txt', \
                                          header=False, index=False)

this leaves 16257 reference TOPMed samples

#### subset the reference panel vcfs to just PD risk regions and exclude samples

In [ ]:
#subset reference panel vcf
#these may be slow operations to put into script file for batch processing

!mkdir -p {WRKDIR}/reference_panels

def subset_loci_from_reference(loci_df, ref_name, in_vcf_name):
    script_name = 'vcf_extract'
    job_file_name = f'{WRKDIR}/{ref_name}.{script_name}.sh'
    with open(job_file_name,'w') as job_file: 
        for chrom in loci_df['chrom'].unique():
            out_vcf = f'{WRKDIR}/reference_panels/{ref_name}.{chrom}.regions.vcf.gz'
            in_vcf = in_vcf_name.format(ref_name, chrom)
            extract_cmd = f'bcftools view --regions-file {WRKDIR}/loci_regions.txt \
--output-file {out_vcf} --output-type z --samples-file ^{WRKDIR}/{ref_name}.excludes.txt \
--force-samples --min-ac 5 {in_vcf}'

            job_file.write(f'{extract_cmd} &\n')
        job_file.write('wait')
        
        print('\n#run these commands at terminal:\n')
        print(f'chmod +x {WRKDIR}/{ref_panel_name}.{script_name}.sh')
        print(f'nohup {WRKDIR}/{ref_panel_name}.{script_name}.sh > \
{WRKDIR}/{ref_panel_name}.{script_name}.log &')
        
#run for ref panel        
ref_panel_name = 'topmed.freeze5b'
ref_vcf_name = '$PATH/datasets/wgs_ref_haplos/hg38/{}.{}.vcf.gz'
subset_loci_from_reference(regions_df, ref_panel_name, ref_vcf_name)


#### ensure the variants ID were renamed with alleles

In [ ]:
#### ensure the variants ID were renamed WITHOUT alleles
def re_id_varaints(loci_df, ref_name):
    script_name = 'vcf_reid'
    job_file_name = f'{WRKDIR}/{ref_name}.{script_name}.sh'
    with open(job_file_name,'w') as job_file:    
        for chrom in loci_df['chrom'].unique():
            out_vcf = f'{WRKDIR}/reference_panels/{ref_name}.{chrom}.regions.reid.vcf.gz'
            in_vcf = f'{WRKDIR}/reference_panels/{ref_name}.{chrom}.regions.vcf.gz'
            annotate_cmd = f'bcftools annotate --output {out_vcf} --output-type z \
--set-id \'%CHROM\:%POS\' {in_vcf}'

            job_file.write(f'{annotate_cmd} &\n')
        job_file.write('wait')            

        print('\n#run these commands at terminal:\n')
        print(f'chmod +x {WRKDIR}/{ref_panel_name}.{script_name}.sh')
        print(f'nohup {WRKDIR}/{ref_panel_name}.{script_name}.sh > \
{WRKDIR}/{ref_panel_name}.{script_name}.log &')

#run for ref panel
ref_panel_name = 'topmed.freeze5b'
re_id_varaints(regions_df, ref_panel_name)


#### make sure dupe variants are removed

In [ ]:
#### make sure dupe variants are removed, causes lots of problems with unlabel LD matrices
def remove_duplicate_varaints(loci_df, ref_name):
    script_name = 'vcf_remove_duplicate_variants'
    job_file_name = f'{WRKDIR}/{ref_name}.{script_name}.sh'
    with open(job_file_name,'w') as job_file:    
        for chrom in loci_df['chrom'].unique():
            out_vcf = f'{WRKDIR}/reference_panels/{ref_name}.{chrom}.regions.reid.nodupes.vcf.gz'
            in_vcf = f'{WRKDIR}/reference_panels/{ref_name}.{chrom}.regions.reid.vcf.gz'
            annotate_cmd = f'bcftools norm --rm-dup all --output {out_vcf} --output-type z {in_vcf}'

            job_file.write(f'{annotate_cmd} &\n')
        job_file.write('wait')            

        print('\n#run these commands at terminal:\n')
        print(f'chmod +x {WRKDIR}/{ref_panel_name}.{script_name}.sh')
        print(f'nohup {WRKDIR}/{ref_panel_name}.{script_name}.sh > \
{WRKDIR}/{ref_panel_name}.{script_name}.log &')

#run for ref panel
ref_panel_name = 'topmed.freeze5b'
remove_duplicate_varaints(regions_df, ref_panel_name)


#### convert reference panel vcfs to plink1 bfiles

In [ ]:
# convert reference panel vcfs to plink1 bfiles
def convert_vcf_plink_bfiles(loci_df, ref_name):
    script_name = 'convert_vcf_plink_bfiles'
    job_file_name = f'{WRKDIR}/{ref_name}.{script_name}.sh'
    with open(job_file_name,'w') as job_file:    
        for chrom in loci_df['chrom'].unique():
            in_vcf = f'{WRKDIR}/reference_panels/{ref_name}.{chrom}.regions.reid.nodupes.vcf.gz'
            out_plink = f'{WRKDIR}/reference_panels/{ref_name}.{chrom}.regions.plink'
            this_cmd = f'plink --vcf {in_vcf} --make-bed --out {out_plink}'

            job_file.write(f'{this_cmd} &\n')
        job_file.write('wait')            

        print('\n#run these commands at terminal:\n')
        !chmod +x {WRKDIR}/{ref_panel_name}.{script_name}.sh
        print(f'nohup {WRKDIR}/{ref_panel_name}.{script_name}.sh > \
{WRKDIR}/{ref_panel_name}.{script_name}.log &')

#run for ref panel
ref_panel_name = 'topmed.freeze5b'
convert_vcf_plink_bfiles(regions_df, ref_panel_name)


#### run GCTA-COJO

GCTA Software tool:
Yang et al. (2011) GCTA: a tool for Genome-wide Complex Trait Analysis. Am J Hum Genet. 88(1): 76-82. [PubMed ID: 21167468]

https://cnsgenomics.com/software/gcta/#COJO

In [ ]:
#pull the tools
!curl --silent -L https://cnsgenomics.com/software/gcta/bin/gcta_1.93.0beta.zip \
    --output {WRKDIR}/tools/gcta_1.93.0beta.zip
    
!unzip {WRKDIR}/tools/gcta_1.93.0beta.zip -d {WRKDIR}/tools/

print('\n#run these commands at terminal:\n')
print(f'sudo cp {WRKDIR}/tools/gcta_1.93.0beta/gcta64 \
/usr/local/bin/gcta64')

In [ ]:
# make the gcta-cojo dir exits

# create the per locus summary files, the meta results were already formated for smr \
# so same format needed for cojo since both are gcta tools
def create_cojo_files(cojo_dir, regions_dict):
    for chrom in regions_dict.keys():
        for this_df in regions_dict[chrom]:
            this_region_results = this_df._to_pandas()
            region_start = this_df['position'].min()
            region_stop = this_df['position'].max()            
            region_name = f'{chrom}-{region_start}-{region_stop}'
            summary_file = f'{cojo_dir}/input/{region_name}.ma'

            #split variant id into postion allele bits
            #reid the variants
            this_region_results['varid'] = this_region_results['chr'] + ':' + \
this_region_results['position'].astype(str)

            this_region_results = this_region_results[['varid', 'A1', 'A2', \
                                                       'freq', 'b', 'se', 'p', 'N']]
            this_region_results.to_csv(summary_file, index=False, sep=' ')

        
#setup the directories    
!mkdir -p {WRKDIR}/cojo/input
!mkdir -p {WRKDIR}/cojo/output

create_cojo_files(f'{WRKDIR}/cojo', regions_by_chrom)

In [ ]:
#now run gcta cojo
script_name = 'run_cojo'

cojo_minp = GWAS_SIG_PVALUE_THRESHOLD
# cojo_minmaf = summary_stats_df['freq'].min()
# go with a much smaller maf, here, assuming all summary stats had appropriate cutoff \
# but the that with the LD ref panel there might be edge cases that are just below threshold
cojo_minmaf = 0.0001
#this will large window won't impact since the summary files are split by the determined regions \
#so just need to make sure full distance of that region is considered in the window \
#but the determined region sizes will likely change the cojo results as some of these will be \
#much larger that 1Mb; 1Mb is not a magical distance threshold anyway
cojo_ldwindow = int(regions_df['size_bp'].max()/1000)

subjob_cnt = 0
with open(f'{WRKDIR}/{script_name}.sh','w') as job_file: 
    for chrom in regions_by_chrom.keys():
        plink_set = f'{WRKDIR}/reference_panels/topmed.freeze5b.{chrom}.regions.plink'
        for this_df in regions_by_chrom[chrom]:
#             this_region_results = this_df._to_pandas()
            this_region_results = this_df.copy()
            region_start = this_df['position'].min()
            region_stop = this_df['position'].max()            
            region_name = f'{chrom}-{region_start}-{region_stop}'
            summary_file = f'{WRKDIR}/cojo/input/{region_name}.ma'
            cojo_out = f'{WRKDIR}/cojo/output/{region_name}'
        
            job_file.write(f'gcta64 --bfile {plink_set} \
--cojo-file {summary_file} --cojo-slct --cojo-p {cojo_minp} --maf {cojo_minmaf} \
--cojo-wind {cojo_ldwindow} --out {cojo_out} > {cojo_out}.log &\n')
            #run 20 of these at a time
            if (subjob_cnt % 20) == 0:
                job_file.write('wait\n')            

    job_file.write('wait')
              
print('\n#run these commands at terminal:\n')
!chmod +x {WRKDIR}/{script_name}.sh
print(f'nohup {WRKDIR}/{script_name}.sh > \
{WRKDIR}/{script_name}.log &') 

NOTE: for chr7-65584435-67544783 I had to go back and lower the cojo-p to 5e-06 to get a result for this region

In [ ]:
# read in the cojo results, and determine causal count per locus
cojo_df = None
regions_df['causal_cnt'] = 0
for row in regions_df.itertuples():
    # load regions cojo results
    this_df = pd.read_csv(f'{WRKDIR}/cojo/output/{row.name}.jma.cojo', sep='\s+')
    causal_cnt = this_df.shape[0]
    regions_df.at[row.Index, 'causal_cnt'] = causal_cnt
    cojo_df = pd.concat([cojo_df, this_df])
#     print(f'{row.name} count: {causal_cnt}', end=' ')
    
print(regions_df.head())
print(cojo_df.shape)
cojo_df.head()

In [ ]:
regions_df['causal_cnt'].describe()

In [ ]:
regions_df['causal_cnt'].sum()

In [ ]:
regions_df.loc[regions_df['causal_cnt'] > 1]

In [ ]:
# save the cojo results
cojo_df.to_csv(f'{WRKDIR}/cojo/results.jma.cojo.csv', index=False)

#### run finemap tool 

Benner C, Havulinna AS, Salomaa V et al. Refining fine-mapping: effect sizes and regional heritability. bioRxiv 2018:318618.

Benner C, Havulinna AS, Järvelin M-R et al. Prospects of Fine-Mapping Trait-Associated Genomic Regions by Using Summary Statistics from Genome-wide Association Studies. Am J Hum Genet 2017, DOI: 10.1016/j.ajhg.2017.08.012.

Benner C, Spencer CCA, Havulinna AS et al. FINEMAP: efficient variable selection using summary data from genome-wide association studies. Bioinformatics 2016;32:1493–501.

http://www.christianbenner.com/finemap_v1.3.1_x86_64.tgz
    

In [ ]:
#pull the tools
!curl --silent -L http://www.christianbenner.com/finemap_v1.3.1_x86_64.tgz \
    --output {WRKDIR}/tools/finemap_v1.3.1_x86_64.tgz
    
!tar -xf {WRKDIR}/tools/finemap_v1.3.1_x86_64.tgz -C {WRKDIR}/tools/

print('\n#run these commands at terminal:\n')
print(f'sudo cp {WRKDIR}/tools/finemap_v1.3.1_x86_64/finemap_v1.3.1_x86_64 \
/usr/local/bin/finemap')

In [ ]:
regions_by_chrom['chr4'][0].head()

#### reformat meta results for finemap analysis

In [ ]:
# finemap uses fileset and control files describing so need to split data 
# by regions
# finemap 1.3.1 with this data always reports max casuals drop back versions for test
# in v1.2 z file switched to including summary stats, specified eff_allele as 2nd allele column
# in v1.3.1 z file summary stat docs does clearly specify which allele should be effect
# the current summary stats file is GCTA format input where 1st allele is effect

## ok so kind of looks like 1.3.1 is working exception for the # of causal snps calc, \
## so can specifiy # independent signal from PD meta cojo and still use finemap PIP and cred set(?)

## current meta result are in COJO-GCTA summary input format
# SNP  - this is the marker name
# A1   - this is the effect allele
# A2   - the other allele
# freq - frequency of the affect allele
# b    - beta, coeff, effect for A1
# se   - overall standard error for effect size estimate
# p    - meta-analysis p-value
# N    - number of sample included in test

def create_finemap_files(fm_dir, regions_dict):
    for chrom in regions_by_chrom.keys():
        for this_df in regions_by_chrom[chrom]:
            this_region_results = this_df._to_pandas()
            region_start = this_df['position'].min()
            region_stop = this_df['position'].max()            
            region_name = f'{chrom}-{region_start}-{region_stop}'
#             print(region_name, end=' ')
            #set job file names
            cntrl_file_name = f'{fm_dir}/input/{region_name}.control'
            z_file = f'{fm_dir}/input/{region_name}.z'
            ld_file = f'{fm_dir}/input/{region_name}.ld'
            snp_file = f'{fm_dir}/output/{region_name}.snp'
            cfg_file = f'{fm_dir}/output/{region_name}.config'
            cred_file = f'{fm_dir}/output/{region_name}.cred'
            log_file = f'{fm_dir}/output/{region_name}.log'


            #split variant id into postion allele bits
            #reid the variants
            this_region_results['varid'] = this_region_results['chr'] + ':' + \
this_region_results['position'].astype(str)

            #calculate the maf based on freq1
            this_region_results['maf'] = np.where(this_region_results['freq'] < 0.50, \
                                                  this_region_results['freq'], \
                                                  1-this_region_results['freq']).round(4)
            
            n_samples = int(this_region_results['N'].mean())

            #gcta expects effect allele first, snptest/finemap expect second
            #this is for v1.3.1
            this_region_results = this_region_results[['varid', 'chr', 'position', \
                                                       'A2', 'A1', 'maf', 'b', 'se']]
            this_region_results.columns = ['rsid', 'chromosome', 'position', 'allele1', \
                                           'allele2', 'maf', 'beta', 'se']

            this_region_results.to_csv(z_file, index=False, sep=' ')

            #this is for v1.3.1
            contro_file_header = 'z;ld;snp;config;cred;n_samples;log'

            with open(cntrl_file_name,'w') as cntrl_file:
                cntrl_file.write(f'{contro_file_header}\n')
                #this is for v1.3.1
                cntrl_file.write(f'{z_file};{ld_file};{snp_file};{cfg_file};\
{cred_file};{n_samples};{log_file}\n')        
        
#setup the directories    
!mkdir -p {WRKDIR}/finemap/input
!mkdir -p {WRKDIR}/finemap/output

create_finemap_files(f'{WRKDIR}/finemap', regions_by_chrom)

#### trim z files so that only variants that are also in the reference are considered
unfortunately this means we are going to loose variants, not great for fine mapping

In [ ]:
#### trim z files so that only variants that are also in the reference are considered
def check_finemap_zfiles(fm_dir, regions_dict, ref_dif):
    missing_counts = []
    
    for chrom in regions_dict.keys():
        #go ahead and load the bim df since we are doing this per chrom not per region on chrom
        try:
            bim_file = f'{ref_dif}/topmed.freeze5b.{chrom}.regions.plink.bim'        
            bim_df = pd.read_csv(bim_file, header=None, sep='\s+')
            bim_set = set(bim_df[1]) #col 1 is varaint names
        except IOError:
            print(f'{chrom} doesn\'t have a file')
            continue
            
        for this_df in regions_dict[chrom]:
            this_region_results = this_df._to_pandas()
            region_start = this_df['position'].min()
            region_stop = this_df['position'].max()            
            region_name = f'{chrom}-{region_start}-{region_stop}'
            z_file = f'{fm_dir}/input/{region_name}.z'

            #load the dataframes
            z_df = pd.read_csv(z_file, sep='\s+')

            lost_var_count = len(set(z_df['rsid']) - bim_set)
            missing_counts.append(lost_var_count)
            
            print(f'{region_name} lost {lost_var_count}', end=' ')
            
            #now remove these missings and save new z file
            z_df = z_df.loc[z_df['rsid'].isin(bim_set)]
            z_df.to_csv(z_file, header=True, index=False, sep=' ')

    print(f'\nmean loss per region {np.mean(missing_counts)}')

#run the function
check_finemap_zfiles(f'{WRKDIR}/finemap', regions_by_chrom, f'{WRKDIR}/reference_panels')            

In [ ]:
#create the ld files named in the created control files
fm_dir = f'{WRKDIR}/finemap'

job_name = 'generate.ld_matrices'
job_file_name = f'{fm_dir}/{job_name}.sh'
with open(job_file_name,'w') as job_file:    
    for chrom in regions_by_chrom.keys():
        for this_df in regions_by_chrom[chrom]:
            this_region_results = this_df._to_pandas()
            region_start = this_df['position'].min()
            region_stop = this_df['position'].max()            
            region_name = f'{chrom}-{region_start}-{region_stop}'

            ld_file = f'{fm_dir}/input/{region_name}'
            z_file = f'{fm_dir}/input/{region_name}.z'
            ref_plink = f'{WRKDIR}/reference_panels/topmed.freeze5b.{chrom}\
.regions.plink'

            job_file.write(f'plink --bfile {ref_plink} --r2 square \
--out {ld_file} --extract {z_file}\n')
            job_file.write(f'sed -i s\"/\\t/ /\"g {ld_file}.ld\n')
        
print('\n#run these commands at terminal:\n')
!chmod +x {job_file_name}
print(f'nohup {job_file_name} > {fm_dir}/{job_name}.log &')
    

In [ ]:
#create script file for running the finemap jobs
script_name = 'run_finemap'

subjob_cnt = 0
with open(f'{WRKDIR}/{script_name}.sh','w') as job_file: 
    for locus_row in regions_df.itertuples():
        subjob_cnt += 1

        cntrl_file_name = f'{fm_dir}/input/{locus_row.name}.control'

        job_file.write(f'finemap --sss --in-files {cntrl_file_name} \
--log --n-causal-snps {locus_row.causal_cnt} &\n')
        #run 20 of these at a time
        if (subjob_cnt % 20) == 0:
            job_file.write('wait\n')
        
    job_file.write('wait')
              
print('\n#run these commands at terminal:\n')
!chmod +x {WRKDIR}/{script_name}.sh
print(f'nohup {WRKDIR}/{script_name}.sh > \
{WRKDIR}/{script_name}.log &') 

In [ ]:
#check expected results are present
print(f'should be {regions_df.shape[0]} of each of these file types')

!ls {fm_dir}/output/*.snp | wc -l
!ls {fm_dir}/output/*.config | wc -l
!ls {fm_dir}/output/*.cred | wc -l
!ls {fm_dir}/output/*.log_sss | wc -l

!less {WRKDIR}/{script_name}.log | grep Error

print('\n#run these commands at terminal:\n')
print(f'ls -lhS {fm_dir}/output/*.snp')

#### parse finemap results to fill in blocks table

In [ ]:
# parse finemap results to fill in blocks table and merge original summary status with finemap PIPs
fm_sumstats_df = None
regions_df['tested_vars'] = 0
regions_df['pip_vars'] = 0
regions_df['h2'] = 0
for chrom in regions_by_chrom.keys():
    print(chrom, end=' ')
    for this_df in regions_by_chrom[chrom]:
        this_region_results = this_df._to_pandas()
        region_start = this_region_results['position'].min()
        region_stop = this_region_results['position'].max()            
        region_name = f'{chrom}-{region_start}-{region_stop}'
        #finemap output files to read
        snp_file = f'{fm_dir}/output/{region_name}.snp'
        cfg_file = f'{fm_dir}/output/{region_name}.config'
        cred_file = f'{fm_dir}/output/{region_name}.cred'
    
        if not os.path.exists(cred_file):
            print(cred_file)
            continue
    
        pip_df = pd.read_csv(snp_file, sep='\s+')

        region_stats_df = this_region_results.loc[this_region_results['position'].isin(pip_df['position'])]
        region_stats_df = region_stats_df.merge(pip_df._to_pandas(), how='inner', \
                                                left_on='position', right_on='position')

        fm_sumstats_df = pd.concat([fm_sumstats_df, region_stats_df])

In [ ]:
# save the full pd_regions fine mapping results
fm_sumstats_df.to_csv(f'{fm_dir}/results.csv', index=False)

In [ ]:
print(fm_sumstats_df.shape)
print(fm_sumstats_df.columns)
fm_sumstats_df.head()

In [ ]:
# update and save the loci summary
for locus in regions_df.itertuples():
    #finemap output files to read
    snp_file = f'{fm_dir}/output/{locus.name}.snp'
    cfg_file = f'{fm_dir}/output/{locus.name}.config'

    pip_df = pd.read_csv(snp_file, sep='\s+')
    cfg_df = pd.read_csv(cfg_file, sep='\s+')
    regions_df.at[locus.Index, 'tested_vars'] = pip_df.shape[0]
    regions_df.at[locus.Index, 'pip_vars'] = pip_df[pip_df['prob'] > 0.01].shape[0]  
    regions_df.at[locus.Index, 'h2'] = cfg_df['h2'].max()
    
#now save updated locus summary
regions_df.to_csv(f'{WRKDIR}/loci_fm_summary.csv', index=False)

In [ ]:
print(regions_df['size_bp'].sum())
print(regions_df['causal_cnt'].sum())
print(regions_df['tested_vars'].sum())
print(regions_df['pip_vars'].sum())
print(regions_df['h2'].sum())

In [ ]:
print(regions_df['size_bp'].describe())
print(regions_df['causal_cnt'].describe())
print(regions_df['tested_vars'].describe())
print(regions_df['pip_vars'].describe())
print(regions_df['h2'].describe())

In [ ]:
regions_df['pip_vars'].sum()

#### plot example local manhattans

In [ ]:
# if skipped down here just read in results from above
fm_sumstats_df = pd.read_csv(f'{WRKDIR}/finemap/results.csv')
print(fm_sumstats_df.shape)
fm_sumstats_df.head()

In [ ]:
#local manhattan plot with p-value and PIP
def lcl_manhattan_plot(chrom, block_start, block_stop, gene_name, gene_start, gene_stop):
    # fm_sumstats_dd = dd.from_pandas(fm_sumstats_df,8)
    this_region_results = fm_sumstats_df.loc[(fm_sumstats_df['chromosome'] == chrom) & \
                                             (fm_sumstats_df['position'] > block_start) & \
                                             (fm_sumstats_df['position'] < block_stop)]

    print(this_region_results.shape)

    plt.figure(figsize=(18,18))
    sns.set_style('dark')

    fig, ax = plt.subplots()
    fig.set_size_inches(12, 8)
    ax2 = ax.twinx()
    
    sns.scatterplot(x=this_region_results['position']/1000, size=abs(this_region_results['z']), \
                    y=np.log10(this_region_results['p'])*-1, color='grey', ax=ax, legend=False)
    
#     y_level = (np.log10(this_region_results['p'])*-1)/2
#     plt.plot([gene_start/1000, gene_stop/1000], [y_level, y_level], linewidth=2)
#     plt.text(gene_start/10002, gene_stop/1000, gene_name)

    sns.scatterplot(x=this_region_results['position']/1000 , size=abs(this_region_results['z']), \
                    y=this_region_results['prob'], color='red', ax=ax2, legend=False)
    # ax2.set_ylim(0,this_region_results['log10bf'].max()+1)
    ax2.set_ylim(0,1.2)

    ax.set(xlabel='chromosome position (kb)', ylabel='-log10(p-value)')
    ax2.set(xlabel='chromosome position (kb)', ylabel='Posterior Inclusion Probability')

    plt.title(f'PD locus {chrom} near {gene_name}',fontsize='large') 
    plt.legend(bbox_to_anchor=(1.01, 1), loc=2, borderaxespad=0,prop={'size': 8})

    plot_out_file_name = f'{WRKDIR}/finemap/images/{chrom}.{gene_name}.local_man.png'
    plt.savefig(plot_out_file_name,format='png',dpi=600,bbox_inches='tight')
    plt.show()


In [ ]:
!mkdir -p '{WRKDIR}/finemap/images'

#INPP5F
chrom = 'chr10'
block_start = 118777099
block_stop = 120776542

gene_name = 'INPP5F'
gene_start = 119726050
gene_stop = 119829147

lcl_manhattan_plot(chrom, block_start, block_stop, gene_name, gene_start, gene_stop)

In [ ]:
#BST1
chrom = 'chr4'
block_start = 14735725
block_stop = 16735661

gene_name = 'BST1'
gene_start = 15703065
gene_stop = 15732787

lcl_manhattan_plot(chrom, block_start, block_stop, gene_name, gene_start, gene_stop)

In [ ]:
#NUCKS1
chrom = 'chr1'
block_start = 204754670
block_stop = 206754269

gene_name = 'NUCKS1'
gene_start = 205712822
gene_stop = 205750182

lcl_manhattan_plot(chrom, block_start, block_stop, gene_name, gene_start, gene_stop)

In [ ]:
#GPNMB
chrom = 'chr7'
block_start = 22260625
block_stop = 24260398

gene_name = 'GPNMB'
gene_start = 23246746
gene_stop = 23275096

lcl_manhattan_plot(chrom, block_start, block_stop, gene_name, gene_start, gene_stop)


In [ ]:
#TMEM175
chrom = 'chr4'
block_start = 65411
block_stop = 1956747

gene_name = 'TMEM175'
gene_start = 932460
gene_stop = 958656

lcl_manhattan_plot(chrom, block_start, block_stop, gene_name, gene_start, gene_stop)


In [ ]:
#SNCA
chrom = 'chr4'
block_start = 88705240
block_stop = 90704927

gene_name = 'SNCA'
gene_start = 89724099
gene_stop = 89838315

lcl_manhattan_plot(chrom, block_start, block_stop, gene_name, gene_start, gene_stop)


In [ ]:
#CTSB
chrom = 'chr8'
block_start = 10855435
block_stop = 12854684

gene_name = 'CTSB'
gene_start = 11842524
gene_stop = 11868087

lcl_manhattan_plot(chrom, block_start, block_stop, gene_name, gene_start, gene_stop)


In [ ]:
#CNTN1
chrom = 'chr12'
block_start = 39341546	
block_stop = 42901508

gene_name = 'LRRK2-CNTN1'
gene_start = 40692442
gene_stop = 41072412

lcl_manhattan_plot(chrom, block_start, block_stop, gene_name, gene_start, gene_stop)


#### compare paper cojo results with these cojo results

In [ ]:
# load the PD meta5v2 Table S2 results
table_s2_df = pd.read_csv(f'{WRKDIR}/table_s2.csv')
print(f'Table S2: {table_s2_df.shape}')

# reload cojo results
cojo_df = pd.read_csv(f'{WRKDIR}/cojo/results.jma.cojo.csv')
print(f'cojo: {cojo_df.shape}')

In [ ]:
%%time
#now pull and map ids for index variants
build = 'hg38'
build_path = f'{WRKDIR}/tools/{build}'

#all the variants of interest are proximal to LRRK2 on chr12
var_positions_df = find_dbsnp_positions(list(table_s2_df['SNP']), \
                                             AUTOSOMES, build_path)
print(var_positions_df.shape)
var_positions_df.head()

In [ ]:
#so looks like some number of summary stats rsIDs are not present in dbSNP151
missing_rsIDs = set(table_s2_df['SNP']) - set(var_positions_df['id'])
print(len(missing_rsIDs))
#33K different in size but only 13K missing lots of dupes?
index_var_stats_missing_pos_df = table_s2_df.loc[table_s2_df['SNP'].\
                                                isin(list(missing_rsIDs))]

print(index_var_stats_missing_pos_df.shape)
print(index_var_stats_missing_pos_df['P'].describe())
print(index_var_stats_missing_pos_df.loc[table_s2_df['P'] < 1e-08].shape)

In [ ]:
# the dbSNP IDs are no longer valid but had significant results
index_var_stats_missing_pos_df.loc[index_var_stats_missing_pos_df['P'] < 1e-08]

In [ ]:
# merge summary stats with variants physical position
table_s2_df = table_s2_df.merge(pd.DataFrame(var_positions_df), \
                                how='inner', left_on='SNP', right_on='id')

table_s2_df['id'] = table_s2_df['chr'] + ':' + table_s2_df['position'].astype('str')
table_s2_df.head()

In [ ]:
len(set(cojo_df['SNP']) & set(table_s2_df['id']))

In [ ]:
# temp_df = table_s2_df._to_pandas()
# display(temp_df.loc[(~temp_df['id'].isin(cojo_df['SNP'])) & \
#                     (temp_df['chr'] == 'chr12')])

display(table_s2_df.loc[(~table_s2_df['id'].isin(cojo_df['SNP'])) & \
                        (table_s2_df['chr'] == 'chr12')])

In [ ]:
#how many of the paper cojos are PIPs here
fm_sumstats_df.shape
pips_df = fm_sumstats_df.loc[fm_sumstats_df['prob'] > 0.01]
print(pips_df.shape)

print(len(set(table_s2_df['SNP']) & set(pips_df['SNP'])))

In [ ]:
#how many of the this analysis cojos are PIPs here
print(len(set(cojo_df['SNP']) & set(pips_df['rsid'])))

In [ ]:
#how many of the these are all three
print(len((set(table_s2_df['SNP']) & set(cojo_df['SNP'])) & set(pips_df['rsid'])))
#huh wasn't what I was necessarily expecting

In [ ]:
#should take a look to see how cojo matches line up with one-signal loci
#but given the zero above probably again not what I'm guessing